# Ajinomoto Project Data Unification
Here we pull in all data types and create a unified data frame with all strain information. This will then be converted into a format that is usable by EDD.

**Todo:**

1. [X] Add IPTG column to metadata
2. [X] Finish Import of Cycle 5 Data
3. [X] Create New Strain ID with B1.3 format for batches
3. [X] Create CSV for Cycle - Strain - JPUB ID for Takuya
4. [X] Create CSV for Pubchem IDs
5. [ ] Put CSVs in Google Drive for Ajinomoto
6. [ ] Send out Email for everyone to fill out their csvs

In [1]:
import pandas as pd
from IPython.display import display

STANDARD_IPTG = 0.1 # mM concentration

## Cycle 1 Import

In [2]:
#Cycle 1 Data Frame Unification 
df = pd.read_csv('data/ajinomoto_oct23.csv')

In [3]:
#Create a Dataframe of Protein and Metabolite Single Time Point Measurement
measurement_df = pd.pivot_table(df, values=['0.0','27.0'], index=['Line Name'], columns=['Protocol Name','Measurement Type']).fillna(0)
measurement_df.columns = measurement_df.columns.droplevel(0)

#Change Proteomics to SwissProt Identifiers
swiss_prot_name = {'A1U2T0_MARHV':'tr|A1U2T0|A1U2T0_MARHV',
                   'A1U3L3_MARHV':'tr|A1U3L3|A1U3L3_MARHV',
                   'ACR1_ACIAD':'sp|Q6F7B8|ACR1_ACIAD',
                   'AHR_ECOLI':'sp|P27250|AHR_ECOLI',
                   'FATB_UMBCA':'sp|Q41635|FATB_UMBCA',
                   'LCFA_ECOLI':'sp|P69451|LCFA_ECOLI'
                  }

chem_names = {'Dodecanoate (n-C12:0)':'dodecan-1-ol'}

#Change GCMS To Correct Naming Convention
measurement_df.rename(columns = swiss_prot_name, inplace = True, level=1)
measurement_df.rename(columns = chem_names, inplace = True, level=1)

#display(measurement_df)

In [4]:
#Load Design data from ICE
design_df = df[['Line Name','Strain']]
design_df = design_df.groupby(['Line Name','Strain']).size().reset_index().set_index('Line Name')[['Strain']]

#Add Batch Number, Tak Number, and Replicate
identifiers = [index.split('-') for index in design_df.index]
identifiers = [[int(identifier[0][1]), int(identifier[1]), int(identifier[2])] for identifier in identifiers]
design_df[['Batch','Strain','Replicate']] = pd.DataFrame(identifiers, index=design_df.index)
design_df['Cycle']=1

design_df.loc[design_df['Strain']==9017,'Cycle'] = 0

#Add Design Information (RBS Strength To Each Design For Each Protein)
rbs_df = pd.read_csv('data/design.csv')
rbs_df.rename(columns = {'Tak':'Strain'}, inplace = True)


#display(rbs_df)
indecies = design_df.index
design_df = pd.merge(design_df,rbs_df,how='left',on="Strain")
design_df.index = indecies

#Add Pathway Variant ID
#design_df['Pathway Variant'] = 0
#pathway_genes = ['Maqu_2507','Maqu_2220','acr1']
#for i,pathway_gene in enumerate(pathway_genes):
    #Find Indecies Where Pathway Gene is present
#    design_df['Pathway Variant'].loc[~design_df[pathway_gene].isnull()] = i+1

design_df.columns = pd.MultiIndex.from_tuples([('Metadata',cname) for cname in design_df.columns])
#display(design_df)

In [5]:
#Load Metabolomics Data
metabolomics_df = pd.read_csv('data/metabolomics.csv')
columns = ['Decanoic acid', 'Lauric acid', 'Myristic acid', 'Palmitic acid',
           'Stearic acid', 'Palmitoleic acid', 'Oleic acid']# 'Dodecanoyl-CoA',
           #'Tetradecanoyl-CoA', 'Hexadecanoyl-CoA', 'Stearoyl-CoA', 'Acetyl-CoA']
augmented_cols = pd.MultiIndex.from_tuples([('LC-MS',col) for col in columns])
metabolomics_df = metabolomics_df.set_index('EDD Line Name')
metabolomics_df = metabolomics_df[columns]
metabolomics_df.columns = augmented_cols
metabolomics_df = metabolomics_df.fillna(0)

In [6]:
#Load Nims Data
nims_df = pd.read_csv('data/nims.csv')
nims_averaged_df = nims_df.groupby('Unnamed: 0').mean()
nims_averaged_df.columns = pd.MultiIndex.from_tuples([('NIMS',col) for col in nims_averaged_df.columns])

In [7]:
df = pd.merge(measurement_df,design_df,how='left',left_index=True,right_index=True)
df = pd.merge(df,metabolomics_df,how='left',left_index=True,right_index=True)
df = pd.merge(df,nims_averaged_df,how='left',left_index=True,right_index=True)

In [8]:
#Load Translation Initiation Rates Associated with Each RBS.
rbs_df = pd.read_csv('data/rbs.csv')
#Define Function to associate Translation Initiation Rate with Each Gene
def find_tir(gene,value_column='Predicted TIR'):
    def f(row):
        rbs_strength = row[('Metadata',gene)]
        if isinstance(rbs_strength,str):
            return rbs_df[(rbs_df['Gene'] == gene)&(rbs_df['Strength'] == row[('Metadata',gene)])][value_column].values[0]
        else:
            return 0        
    return f

genes = ['UcFatB1','FadD','Maqu_2507','Maqu_2220','acr1','yjgB']
for gene in genes:
    df[('TIR',gene)] = df.apply(find_tir(gene),axis=1)
    df[('EMOPEC',gene)] = df.apply(find_tir(gene,value_column='EMOPEC'),axis=1)
    measurement_df[('TIR',gene)] = df.apply(find_tir(gene),axis=1)
    measurement_df[('EMOPEC',gene)] = df.apply(find_tir(gene,value_column='EMOPEC'),axis=1)

replacement = {'UcFatB1':'sp|Q41635|FATB_UMBCA',
               'FadD':'sp|P69451|LCFA_ECOLI',
               'Maqu_2507':'tr|A1U3L3|A1U3L3_MARHV',
               'Maqu_2220':'tr|A1U2T0|A1U2T0_MARHV',
               'acr1':'sp|Q6F7B8|ACR1_ACIAD',
               'yjgB':'sp|P27250|AHR_ECOLI'}

df.rename(columns = replacement, inplace = True, level=1)

In [9]:
#drop some metadata columns
proteins = ['sp|P27250|AHR_ECOLI','sp|P69451|LCFA_ECOLI','sp|Q41635|FATB_UMBCA','sp|Q6F7B8|ACR1_ACIAD','tr|A1U2T0|A1U2T0_MARHV','tr|A1U3L3|A1U3L3_MARHV']
for col in proteins:
    df = df.drop(('Metadata',col),axis=1)
#display(df['Metadata'])
df[('Metadata','IPTG')] = STANDARD_IPTG

## Import Cycle 2 Data

In [10]:
#Import Batch3
#Import Data Files
cycle2_df = pd.read_csv('data/Cycle2Batch3.csv')
cycle2_df =  pd.pivot_table(cycle2_df, values=['27'], index=['Line Name'], columns=['Protocol Name','Measurement Type']).fillna(0)
cycle2_df.columns = cycle2_df.columns.droplevel(0)

#Add Metadata
design_df = pd.DataFrame()
identifiers = [index.split('-') for index in cycle2_df.index]
identifiers = [[3,identifier[0],int(identifier[1][1:])] for identifier in identifiers]

import re

def id_to_metadata(ID):
    cycle,strain = re.split(r"[_, ]", ID)
    cycle = int(cycle[1])
    strain = int(re.sub('[^0-9]','', strain))
    
    if strain==9017:
        cycle = 0
    return cycle,strain
    
design_df['Cycle'] = 0
design_df['Strain'] = 0
design_df['IPTG'] = STANDARD_IPTG
design_df[['Batch','ID','Replicate']] = pd.DataFrame(identifiers, index=cycle2_df.index)
design_df.columns = pd.MultiIndex.from_tuples([('Metadata',cname) for cname in design_df.columns])
design_df[('Metadata','Cycle')] = design_df[('Metadata','ID')].apply(lambda x: id_to_metadata(x)[0])
design_df[('Metadata','Strain')] = design_df[('Metadata','ID')].apply(lambda x: id_to_metadata(x)[1])
design_df = design_df.drop(('Metadata', 'ID'), axis = 1)

#Merge DataFrame
cycle2_df = pd.merge(cycle2_df,design_df,how='left',right_index=True,left_index=True)

#Convert Targeted Proteomics to Swiss Prot
cycle2_df.rename(columns = {'AHR_ECOLI':'sp|P27250|AHR_ECOLI','LCFA_ECOLI':'sp|P69451|LCFA_ECOLI'}, inplace = True, level=1)


In [11]:
#Import Batch4
batch4_df = pd.read_csv('data/Cycle2Batch4.csv')
batch4_df =  pd.pivot_table(batch4_df, values=['27.0'], index=['Line Name'], columns=['Protocol Name','Measurement Type']).fillna(0)
batch4_df.columns = batch4_df.columns.droplevel(0)

#Add Metadata
design_df = pd.DataFrame()
identifiers = [index.split('-') for index in batch4_df.index]
identifiers = [[int(identifier[1][1:]),identifier[2],int(identifier[3][1:])] for identifier in identifiers]
design_df[['Batch','ID','Replicate']] = pd.DataFrame(identifiers, index=batch4_df.index)
design_df['Cycle'] = design_df['ID'].apply(lambda x: int(x.split('_')[1][1]))
design_df['Strain'] = design_df['ID'].apply(lambda x: int(x.split('_')[0]))
design_df['IPTG'] = STANDARD_IPTG
design_df = design_df.drop('ID',axis=1)
design_df.columns = pd.MultiIndex.from_tuples([('Metadata',cname) for cname in design_df.columns])

batch4_df = pd.merge(batch4_df,design_df,how='left',right_index=True,left_index=True)
batch4_df.rename(columns = {'AHR_ECOLI':'sp|P27250|AHR_ECOLI','LCFA_ECOLI':'sp|P69451|LCFA_ECOLI'}, inplace = True, level=1)
display(batch4_df)

Protocol Name           GC-MS Targeted Proteomics                       \
Measurement Type dodecan-1-ol sp|P27250|AHR_ECOLI sp|P69451|LCFA_ECOLI   
Line Name                                                                
c2-b4-01_c2-R1        0.33696              1935.0             217353.0   
c2-b4-01_c2-R2        0.35211              1407.0             231424.0   
c2-b4-01_c2-R3        0.37076              2469.0             235110.0   
c2-b4-02_c2-R1        0.00000              1970.0             285965.0   
c2-b4-02_c2-R2        0.00000              2230.0             294208.0   
c2-b4-02_c2-R3        0.00000              1964.0             296409.0   
c2-b4-04_c2-R1        0.24434              2000.0             247382.0   
c2-b4-04_c2-R2        0.25846              1701.0             236865.0   
c2-b4-04_c2-R3        0.27516              1849.0             251422.0   
c2-b4-11_c1-R1        0.68394              1184.0             217404.0   
c2-b4-11_c1-R2        0.69810              2373.0             219170.0   
c2-b4-11_c1-R3        0.67880              2177.0             211659.0   
c2-b4-11_c2-R1        0.16912              4359.0             269720.0   
c2-b4-11_c2-R2        0.17383              2978.0             246862.0   
c2-b4-11_c2-R3        0.20522              2836.0             229763.0   
c2-b4-14_c2-R1        0.67963              1575.0             234896.0   
c2-b4-14_c2-R2        0.65743              1353.0             201418.0   
c2-b4-14_c2-R3        0.63307               866.0             226752.0   
c2-b4-24_c2-R1        0.00000              6290.0               1865.0   
c2-b4-24_c2-R2        0.00000              4330.0                178.0   
c2-b4-24_c2-R3        0.00000              4773.0               5650.0   

Protocol Name                                               \
Measurement Type sp|Q41635|FATB_UMBCA sp|Q6F7B8|ACR1_ACIAD   
Line Name                                                    
c2-b4-01_c2-R1               175485.0                  0.0   
c2-b4-01_c2-R2               194337.0                  0.0   
c2-b4-01_c2-R3               189740.0                  0.0   
c2-b4-02_c2-R1                22397.0                  0.0   
c2-b4-02_c2-R2                25199.0                887.0   
c2-b4-02_c2-R3                17736.0               1135.0   
c2-b4-04_c2-R1               280000.0                  0.0   
c2-b4-04_c2-R2               272740.0               2353.0   
c2-b4-04_c2-R3               303288.0                  0.0   
c2-b4-11_c1-R1               105854.0                  0.0   
c2-b4-11_c1-R2               116151.0                  0.0   
c2-b4-11_c1-R3               117200.0                632.0   
c2-b4-11_c2-R1               250711.0               1224.0   
c2-b4-11_c2-R2               228708.0                  0.0   
c2-b4-11_c2-R3               213330.0                  0.0   
c2-b4-14_c2-R1               159139.0                  0.0   
c2-b4-14_c2-R2               156355.0                  0.0   
c2-b4-14_c2-R3               159985.0                  0.0   
c2-b4-24_c2-R1                43929.0                  0.0   
c2-b4-24_c2-R2                47381.0                795.0   
c2-b4-24_c2-R3                42775.0               1574.0   

Protocol Name                                                  Metadata  \
Measurement Type tr|A1U2T0|A1U2T0_MARHV tr|A1U3L3|A1U3L3_MARHV    Batch   
Line Name                                                                 
c2-b4-01_c2-R1                  10196.0              1518587.0        4   
c2-b4-01_c2-R2                   1896.0              1546227.0        4   
c2-b4-01_c2-R3                    528.0              1528867.0        4   
c2-b4-02_c2-R1                   5618.0              1629038.0        4   
c2-b4-02_c2-R2                  18468.0              1690829.0        4   
c2-b4-02_c2-R3                   5642.0              1809021.0        4   
c2-b4-04_c2-R1                  15026.0              154561

In [12]:
#Import Batch4
batch5_df = pd.read_csv('data/Cycle2Batch5.csv')
batch5_df =  pd.pivot_table(batch5_df, values=['27.0'], index=['Line Name'], columns=['Protocol Name','Measurement Type']).fillna(0)
batch5_df.columns = batch5_df.columns.droplevel(0)

#display(batch5_df)

#Add Metadata
design_df = pd.DataFrame()
identifiers = [index.split('-') for index in batch5_df.index]
identifiers = [[int(identifier[1][1:]),identifier[2],int(identifier[3][1:])] for identifier in identifiers]
design_df[['Batch','ID','Replicate']] = pd.DataFrame(identifiers, index=batch5_df.index)
design_df['Strain'] = design_df['ID'].apply(lambda x: int(x.split('_')[0]))
#display(design_df['ID'].apply(lambda x: x.split('_')))

def extract_cycle(id_string):
    id_elements = id_string.split('_')
    if len(id_elements) == 1:
        return 0
    else:
        return int(id_elements[1][1])

def get_iptg(id_string):
    id_elements = id_string.split('_')
    if len(id_elements) == 3:
        return id_elements[2][:3]
    else:
        return STANDARD_IPTG

design_df['Cycle'] = design_df['ID'].apply(extract_cycle)
design_df['IPTG'] = design_df['ID'].apply(get_iptg)

design_df = design_df.drop('ID',axis=1)
design_df.columns = pd.MultiIndex.from_tuples([('Metadata',cname) for cname in design_df.columns])

batch5_df = pd.merge(batch5_df,design_df,how='left',right_index=True,left_index=True)
batch5_df.rename(columns = {'AHR_ECOLI':'sp|P27250|AHR_ECOLI','LCFA_ECOLI':'sp|P69451|LCFA_ECOLI'}, inplace = True, level=1)
display(batch5_df)

Protocol Name                   GC-MS Metadata                            
Measurement Type         dodecan-1-ol    Batch Replicate Strain Cycle IPTG
Line Name                                                                 
c2-b5-01_c2_0.5mMiptg-R1      0.27883        5         1      1     2  0.5
c2-b5-01_c2_0.5mMiptg-R2      0.28538        5         2      1     2  0.5
c2-b5-01_c2_0.5mMiptg-R3      0.27344        5         3      1     2  0.5
c2-b5-01_c2_1.0mMiptg-R1      0.30908        5         1      1     2  1.0
c2-b5-01_c2_1.0mMiptg-R2      0.31091        5         2      1     2  1.0
c2-b5-01_c2_1.0mMiptg-R3      0.29577        5         3      1     2  1.0
c2-b5-02_c2_0.5mMiptg-R1      0.00000        5         1      2     2  0.5
c2-b5-02_c2_0.5mMiptg-R2      0.00000        5         2      2     2  0.5
c2-b5-02_c2_0.5mMiptg-R3      0.00000        5         3      2     2  0.5
c2-b5-02_c2_1.0mMiptg-R1      0.00000        5         1      2     2  1.0
c2-b5-02_c2_1.0mMiptg-R2      0.00000        5         2      2     2  1.0
c2-b5-02_c2_1.0mMiptg-R3      0.00579        5         3      2     2  1.0
c2-b5-03_c2-R1                0.00000        5         1      3     2  0.1
c2-b5-03_c2-R2                0.00000        5         2      3     2  0.1
c2-b5-03_c2-R3                0.00000        5         3      3     2  0.1
c2-b5-03_c2_0.5mMiptg-R1      0.00000        5         1      3     2  0.5
c2-b5-03_c2_0.5mMiptg-R2      0.00000        5         2      3     2  0.5
c2-b5-03_c2_0.5mMiptg-R3      0.00000        5         3      3     2  0.5
c2-b5-03_c2_1.0mMiptg-R1      0.00000        5         1      3     2  1.0
c2-b5-03_c2_1.0mMiptg-R2      0.00000        5         2      3     2  1.0
c2-b5-03_c2_1.0mMiptg-R3      0.00000        5         3      3     2  1.0
c2-b5-04_c2_0.5mMiptg-R1      0.24766        5         1      4     2  0.5
c2-b5-04_c2_0.5mMiptg-R2      0.24679        5         2      4     2  0.5
c2-b5-04_c2_0.5mMiptg-R3      0.26410        5         3      4     2  0.5
c2-b5-04_c2_1.0mMiptg-R1      0.23005        5         1      4     2  1.0
c2-b5-04_c2_1.0mMiptg-R2      0.24922        5         2      4     2  1.0
c2-b5-04_c2_1.0mMiptg-R3      0.27461        5         3      4     2  1.0
c2-b5-07_c2-R1                0.93985        5         1      7     2  0.1
c2-b5-07_c2-R2                0.92886        5         2      7     2  0.1
c2-b5-07_c2-R3                0.94019        5         3      7     2  0.1
c2-b5-09_c2-R1                0.14078        5         1      9     2  0.1
c2-b5-09_c2-R2                0.15048        5         2      9     2  0.1
c2-b5-09_c2-R3                0.14510        5         3      9     2  0.1
c2-b5-11_c1-R1                0.72920        5         1     11     1  0.1
c2-b5-11_c1-R2                0.74988        5         2     11     1  0.1
c2-b5-11_c1-R3                0.73598        5         3     11     1  0.1
c2-b5-11_c1_0.5mMiptg-R1      0.65291        5         1     11     1  0.5
c2-b5-11_c1_0.5mMiptg-R2      0.66226        5         2     11     1  0.5
c2-b5-11_c1_0.5mMiptg-R3      0.64032        5         3     11     1  0.5
c2-b5-18_c2-R1                0.03711        5         1     18     2  0.1
c2-b5-18_c2-R2                0.01411        5         2     18     2  0.1
c2-b5-18_c2-R3                0.01207        5         3     18     2  0.1
c2-b5-9017-R1                 0.00000        5         1   9017     0  0.1
c2-b5-9017-R2                 0.00000        5         2   9017     0  0.1
c2-b5-9017-R3                 0.00000        5         3   9017     0  0.1

In [13]:
#Merge Together Both Batch 3 & 4
cycle2_df = pd.concat([cycle2_df,batch4_df,batch5_df])

## Merge Cycle 1 & 2 Data

In [14]:
cycle2_df[('Metadata','Batch')] += 1

In [15]:
df = pd.concat([df,cycle2_df])
#display(df.loc[df[('Metadata','Cycle')]==2]['TIR'])

In [16]:
#Remove Control Strains with RFP
df = df.drop(df.loc[df[('Metadata','Cycle')].isnull()].index)
#display(df.loc[df[('Metadata','Cycle')].isnull()])

In [17]:
#Create New Standardized Line Names
#Line Name Format C1-S9-B3-R1

df = df.reset_index()

def rename_line(x):
    
    #Get an updataed batch string
    if x[('Metadata','Batch')] < 4:
        batch_string = '1.{}'.format(int(x[('Metadata','Batch')]))
    else:
        batch_string = '2.{}'.format(int(x[('Metadata','Batch')])-1)
        
    line_name = 'C{}-S{}-B{}-R{}'.format(int(x[('Metadata','Cycle')]),
                                         int(x[('Metadata','Strain')]),
                                         batch_string,
                                         int(x[('Metadata','Replicate')]))
                             
    #Check to add IPTG for strains that do not have standard concentration
    if float(x[('Metadata','IPTG')]) > STANDARD_IPTG:
        line_name += '-IPTG{}'.format(x[('Metadata','IPTG')])
        
    return line_name
                                               
df['Line Name'] = df.apply(rename_line,axis=1)
df = df.set_index('Line Name')
display(df)

Protocol Name                       EMOPEC                       \
Measurement Type       sp|P27250|AHR_ECOLI sp|P69451|LCFA_ECOLI   
Line Name                                                         
C1-S1-B1.1-R1                          0.0                 14.8   
C1-S1-B1.1-R2                          0.0                 14.8   
C1-S2-B1.1-R1                          0.0                 92.4   
C1-S2-B1.1-R2                          0.0                 92.4   
C1-S4-B1.1-R1                          0.0                 14.8   
C1-S4-B1.1-R2                          0.0                 14.8   
C1-S8-B1.1-R1                          0.0                 92.4   
C1-S8-B1.1-R2                          0.0                 92.4   
C1-S9-B1.1-R1                          0.0                 87.6   
C1-S9-B1.1-R2                          0.0                 87.6   
C1-S11-B1.1-R1                         0.0                 92.4   
C1-S11-B1.1-R2                         0.0                 92.4   
C1-S12-B1.1-R1                         0.0                 87.6   
C1-S12-B1.1-R2                         0.0                 87.6   
C1-S15-B1.1-R1                         0.0                 14.8   
C1-S15-B1.1-R2                         0.0                 14.8   
C1-S16-B1.1-R1                         0.0                 92.4   
C1-S16-B1.1-R2                         0.0                 92.4   
C1-S22-B1.1-R1                        41.1                 92.4   
C1-S22-B1.1-R2                        41.1                 92.4   
C0-S9017-B1.1-R1                       0.0                  0.0   
C0-S9017-B1.1-R2                       0.0                  0.0   
C1-S11-B1.2-R1                         0.0                 92.4   
C1-S11-B1.2-R2                         0.0                 92.4   
C1-S11-B1.2-R3                         0.0                 92.4   
C1-S19-B1.2-R1                         0.0                 92.4   
C1-S19-B1.2-R2                         0.0                 92.4   
C1-S19-B1.2-R3                         0.0                 92.4   
C1-S25-B1.2-R1                        41.1                 14.8   
C1-S25-B1.2-R2                        41.1                 14.8   
...                                    ...                  ...   
C2-S3-B2.5-R1-IPTG0.5                  NaN                  NaN   
C2-S3-B2.5-R2-IPTG0.5                  NaN                  NaN   
C2-S3-B2.5-R3-IPTG0.5                  NaN                  NaN   
C2-S3-B2.5-R1-IPTG1.0                  NaN                  NaN   
C2-S3-B2.5-R2-IPTG1.0                  NaN                  NaN   
C2-S3-B2.5-R3-IPTG1.0                  NaN                  NaN   
C2-S4-B2.5-R1-IPTG0.5                  NaN                  NaN   
C2-S4-B2.5-R2-IPTG0.5                  NaN                  NaN   
C2-S4-B2.5-R3-IPTG0.5                  NaN                  NaN   
C2-S4-B2.5-R1-IPTG1.0                  NaN                  NaN   
C2-S4-B2.5-R2-IPTG1.0                  NaN                  NaN   
C2-S4-B2.5-R3-IPTG1.0                  NaN                  NaN   
C2-S7-B2.5-R1                          NaN                  NaN   
C2-S7-B2.5-R2                          NaN                  NaN   
C2-S7-B2.5-R3                          NaN                  NaN   
C2-S9-B2.5-R1                          NaN                  NaN   
C2-S9-B2.5-R2                          NaN                  NaN   
C2-S9-B2.5-R3                          NaN                  NaN   
C1-S11-B2.5-R1                         NaN                  NaN   
C1-S11-B2.5-R2                         NaN                  NaN   
C1-S11-B2.5-R3                         NaN                  NaN   
C1-S11-B2.5-R1-IPTG0.5                 NaN                  NaN   
C1-S11-B2.5-R2-IPTG0.5                 NaN                  NaN   
C1-S11-B2.5-R3-IPTG0.5                 NaN                  NaN   
C2-S18-B2.5-R1                         NaN                  NaN   
C2-S18-B2.5-R2                         NaN     

In [18]:
#Add Time Column (Measurement Time 27 Hours)
df[('Metadata','Time')] = 27

In [19]:
#Load RBS DataFrame
rbs_df = pd.read_csv('data/Cycle2RBSPredictions.csv')
rbs_df['Strain'] = rbs_df['ID'].apply(lambda x: int(x.split('-')[-1]))
#rbs_df = rbs_df.set_index('Strain')

#Fill in all Cycle 2 strains with Base Strain Data
#Add TIR & EMOPEC Values to dataframe for Cycle 2
find_expression = lambda x: df.loc[(df[('Metadata','Cycle')]==1)&(df[('Metadata','Strain')]==x)]['EMOPEC'].iloc[0,:]

merge_df = df.loc[(df[('Metadata','Cycle')]==1)&(df[('Metadata','Strain')].isin(rbs_df['Base Strain']))&(df[('Metadata','Batch')]<4),
                   df.columns.get_level_values(0).isin(['EMOPEC','TIR','Metadata'])].groupby([('Metadata','Strain')]).first().reset_index()

rbs_df.columns = pd.MultiIndex.from_product([['Base'],rbs_df.columns])

#Convert Columns to Numerical Values
rbs_df[('Base','Base Strain')] = rbs_df[('Base','Base Strain')].apply(lambda x: int(x))
merge_df[('Metadata','Strain')] = merge_df[('Metadata','Strain')].apply(lambda x: int(x))

#Merge TIR and EMOPEC Data into rbs dataframe
rbs_df= pd.merge(rbs_df,merge_df,how='left',left_on=[('Base','Base Strain')],right_on=[('Metadata','Strain')])

#Add updated strain TIR & Emopec to each strain
def update_rbs_df(row):
    #Update Metadata
    row[('Metadata','Cycle')] = 2
    row[('Metadata','Strain')] = row[('Base','Strain')]
    
    #Update TIR & EMOPEC
    if row[('Base','Gene')] == 'FatB':
        row[('EMOPEC','sp|Q41635|FATB_UMBCA')] = row[('Base','EMOPEC')]
        row[('TIR','sp|Q41635|FATB_UMBCA')] = row[('Base','TIR')]
        
    elif row[('Base','Gene')] == 'FadD':
        row[('EMOPEC','sp|P69451|LCFA_ECOLI')] = row[('Base','EMOPEC')]
        row[('TIR','sp|P69451|LCFA_ECOLI')] = row[('Base','TIR')]
        
    return row

#Update Dataframe
rbs_df = rbs_df.apply(update_rbs_df,axis=1)

#Drop columns that will interfere with merge (Batch/Replicate) + Base
rbs_df = rbs_df.drop(df.loc[:,[('Metadata','Batch'),('Metadata','Replicate')]].head(0).columns, axis=1)
rbs_df = rbs_df.drop(rbs_df.loc[:,rbs_df.columns.get_level_values(0)=='Base'].head(0).columns,axis=1)
#display(rbs_df)

#Merge rbs_df back into the cycle2 dataframe
df[('Metadata','Strain')] = df[('Metadata','Strain')].apply(lambda x: int(x))
rbs_df[('Metadata','Strain')] = rbs_df[('Metadata','Strain')].apply(lambda x: int(x))

#display(df['Metadata'])
#merge_df = pd.merge(df.loc[df[('Metadata','Cycle')]==2],rbs_df,how='inner',on=[('Metadata','Strain')])
#display(merge_df)

def combine_rbs(row,df):
    #display(row)
    try:
        row['EMOPEC'] = df.loc[df[('Metadata','Strain')]==row[('Metadata','Strain')]]['EMOPEC'].iloc[0]
        row['TIR'] = df.loc[df[('Metadata','Strain')]==row[('Metadata','Strain')]]['TIR'].iloc[0]
    except:
        pass
    return row

df.loc[df[('Metadata','Cycle')]==2] = df.loc[df[('Metadata','Cycle')]==2].apply(lambda x: combine_rbs(x,rbs_df),axis=1)
#display(df.loc[df[('Metadata','Cycle')]==2])

#Update All cycle 1 Strains that don't have EMOPEC & 
df.loc[(df[('Metadata','Batch')]>3)&(df['Metadata','Cycle']<2)] = df.loc[(df[('Metadata','Batch')]>3)&(df['Metadata','Cycle']<2)].apply(lambda x: combine_rbs(x,df),axis=1)

#Display Remaining TIR/EMOPEC Values that are not filled in.
display(df.loc[df[('EMOPEC','sp|P27250|AHR_ECOLI')].isnull()]['Metadata'])

Measurement Type,Batch,Cycle,IPTG,Replicate,Strain,Time
Line Name,,,,,,
C2-S10-B2.3-R1,4.0,2.0,NaN,1.0,10,27
C2-S10-B2.3-R2,4.0,2.0,NaN,2.0,10,27
C2-S10-B2.3-R3,4.0,2.0,NaN,3.0,10,27
C2-S11-B2.3-R1,4.0,2.0,NaN,1.0,11,27
C2-S11-B2.3-R2,4.0,2.0,NaN,2.0,11,27
C2-S11-B2.3-R3,4.0,2.0,NaN,3.0,11,27
C2-S11-B2.4-R1,5.0,2.0,0.1,1.0,11,27
C2-S11-B2.4-R2,5.0,2.0,0.1,2.0,11,27
C2-S11-B2.4-R3,5.0,2.0,0.1,3.0,11,27


In [20]:
#Add Pathway Data to Metadata
print(max(float('NaN'),float('NaN')))
import numpy as np
import math
def get_pathway(row):
    pathway_genes = [row[('TIR','tr|A1U2T0|A1U2T0_MARHV')],row[('TIR','tr|A1U3L3|A1U3L3_MARHV')],row[('TIR','sp|Q6F7B8|ACR1_ACIAD')]]
    if max(pathway_genes) == 0:
        return 0
    elif math.isnan(max(pathway_genes)):
        return float('NaN')
    else:
        return np.argmax(pathway_genes) + 1
    
    return np.argmax(row[('')])

df[('Metadata','Pathway')] = df.apply(get_pathway,axis=1)
print(df[('Metadata','Pathway')].unique())
display(df['Metadata'])

nan
[ 2.  1.  3.  0. nan]


Measurement Type,Batch,Cycle,IPTG,Replicate,Strain,Time,Pathway
Line Name,,,,,,,
C1-S1-B1.1-R1,1.0,1.0,0.1,1.0,1,27,2.0
C1-S1-B1.1-R2,1.0,1.0,0.1,2.0,1,27,2.0
C1-S2-B1.1-R1,1.0,1.0,0.1,1.0,2,27,2.0
C1-S2-B1.1-R2,1.0,1.0,0.1,2.0,2,27,2.0
C1-S4-B1.1-R1,1.0,1.0,0.1,1.0,4,27,2.0
C1-S4-B1.1-R2,1.0,1.0,0.1,2.0,4,27,2.0
C1-S8-B1.1-R1,1.0,1.0,0.1,1.0,8,27,2.0
C1-S8-B1.1-R2,1.0,1.0,0.1,2.0,8,27,2.0
C1-S9-B1.1-R1,1.0,1.0,0.1,1.0,9,27,2.0


In [21]:
print(list(df.columns))

[('EMOPEC', 'sp|P27250|AHR_ECOLI'), ('EMOPEC', 'sp|P69451|LCFA_ECOLI'), ('EMOPEC', 'sp|Q41635|FATB_UMBCA'), ('EMOPEC', 'sp|Q6F7B8|ACR1_ACIAD'), ('EMOPEC', 'tr|A1U2T0|A1U2T0_MARHV'), ('EMOPEC', 'tr|A1U3L3|A1U3L3_MARHV'), ('GC-MS', 'Hexadecanoate (n-C16:0)'), ('GC-MS', 'Octadecanoate (n-C18:0)'), ('GC-MS', 'Tetradecanoate (n-C14:0)'), ('GC-MS', 'dodecan-1-ol'), ('HPLC', 'Acetate'), ('HPLC', 'D-Glucose'), ('HPLC', 'D-Lactate'), ('HPLC', 'Formate'), ('HPLC', 'Pyruvate'), ('HPLC', 'Succinate'), ('LC-MS', '(S)-3-Hydroxyhexadecanoyl-CoA'), ('LC-MS', '3-Phospho-D-glycerate'), ('LC-MS', 'ADP'), ('LC-MS', 'AMP'), ('LC-MS', 'ATP'), ('LC-MS', 'Acetyl-CoA'), ('LC-MS', 'Citrate'), ('LC-MS', 'D-Fructose 1,6-bisphosphate'), ('LC-MS', 'D-Fructose 6-phosphate'), ('LC-MS', 'D-Glucose 6-phosphate'), ('LC-MS', 'Decanoic acid'), ('LC-MS', 'Dihydroxyacetone phosphate'), ('LC-MS', 'Dodecanoyl-CoA (n-C12:0CoA)'), ('LC-MS', 'Flavin adenine dinucleotide oxidized'), ('LC-MS', 'Glyceraldehyde 3-phosphate'), ('LC-M

### Write out CSV and Verify it!
Write out CSVs for the complete aggrigate data set (Which I will use).  Additionally Write out EDD specific Files for upload and use By Ajinomoto People

In [22]:
#Write out Complete Data (EDD_Ajinomoto_Data)
df.to_csv('data/EDD_Ajinomoto_Data.csv')

In [23]:
#Make sure it can be pulled in easily
df_check = pd.read_csv('data/EDD_Ajinomoto_Data.csv',header=[0, 1], index_col=0,skipinitialspace=False)
#display(df_check['Metadata'])

In [24]:
#Create Takuya CSV
tak_df = df[[('Metadata','Cycle'),('Metadata','Strain')]]
tak_df[('Metadata','JPUB')] = float('NaN')
tak_df = tak_df.groupby([('Metadata','Cycle'),('Metadata','Strain')]).size().reset_index()
tak_df.columns = ['Cycle','Strain','Count']
tak_df['JPUB'] = float('NaN')

tak_df[['Cycle','Strain','JPUB']].to_csv('data/takuya_jpub.csv',index=False)
display(tak_df)

/Users/zakcostello/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Cycle,Strain,Count,JPUB
0,0.0,9017,14,NaN
1,1.0,1,5,NaN
2,1.0,2,5,NaN
3,1.0,3,3,NaN
4,1.0,4,2,NaN
5,1.0,5,3,NaN
6,1.0,6,3,NaN
7,1.0,7,3,NaN
8,1.0,8,5,NaN
9,1.0,9,2,NaN


In [25]:
#PubChemID
#print(list(df.columns.get_level_values(0).unique()))
pubchem_df = pd.DataFrame(list(df.loc[:,df.columns.get_level_values(0).isin(['LC-MS','HPLC','GC-MS','NIMS'])].columns.get_level_values(1).unique()))

pubchem_df.columns = ['Curent Name']
pubchem_df['PubChem ID'] = float('NaN')
pubchem_df = pubchem_df.set_index('Curent Name')
pubchem_df.to_csv('data/pubchem_names.csv')
print(len(pubchem_df))
display(pubchem_df)

51


,PubChem ID
Curent Name,
Hexadecanoate (n-C16:0),NaN
Octadecanoate (n-C18:0),NaN
Tetradecanoate (n-C14:0),NaN
dodecan-1-ol,NaN
Acetate,NaN
D-Glucose,NaN
D-Lactate,NaN
Formate,NaN
Pyruvate,NaN
